**Aim:** Write a program to exhibit the decision tree based CART Algorithm <br>
**Theory:** The CART algorithm is structured as a sequence of questions, the answers to which determine what the next question, if any should be.  The result of these questions is a tree like structure where the ends are terminal nodes at which point there are no more questions.  A simple example of a decision tree is as follows <br>
The main elements of CART (and any decision tree algorithm) are:
Rules for splitting data at a node based on the value of one variable;
Stopping rules for deciding when a branch is terminal and can be split no more; and
Finally, a prediction for the target variable in each terminal node.



**Code:**<br>
Dataset used is **Iris Dataset**

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder 
import math
from collections import Counter
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('datasets_19_420_Iris.csv')
df=df.drop('Id',axis=1)

df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
le=LabelEncoder()
df['Species']=le.fit_transform(df['Species'])
df['Species'].value_counts()

2    50
1    50
0    50
Name: Species, dtype: int64

In [4]:
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
df.groupby(['SepalLengthCm']).head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
141,6.9,3.1,5.1,2.3,2
143,6.8,3.2,5.9,2.3,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [6]:
X=df.drop(columns=['Species'])
y=df.drop(columns=['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm'])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [8]:
x_final_df= pd.concat([X_train,y_train],axis=1)
x_final_df.shape

(127, 5)

# CART Implementation

In [9]:
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

In [10]:
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
    if row[index] < value:
            left.append(row)
        else:
        right.append(row)
    return left, right

In [26]:
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}


In [12]:
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

In [13]:
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

In [14]:
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

In [15]:
tree = build_tree(np.array(x_final_df), 10, 15)

In [16]:
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [17]:
print_tree(tree)

[X3 < 3.000]
 [X1 < 4.800]
  [0.0]
  [X1 < 4.800]
   [0.0]
   [0.0]
 [X4 < 1.800]
  [X3 < 5.000]
   [X4 < 1.700]
    [X1 < 6.700]
     [X1 < 6.400]
      [X1 < 6.000]
       [X1 < 5.500]
        [1.0]
        [1.0]
       [1.0]
      [1.0]
     [1.0]
    [2.0]
   [2.0]
  [X3 < 4.900]
   [2.0]
   [X1 < 6.400]
    [2.0]
    [X1 < 6.400]
     [2.0]
     [2.0]


In [18]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [19]:
x_final_test=pd.concat([X_test,y_test],axis=1)
x_final_test=np.array(x_final_test)

In [22]:
sumk=0
stump = {'index': 0, 'right': 1, 'value': 4, 'left': 0}
for row in x_final_test:
    prediction=predict(stump,row)
    if prediction==int(row[-1]):
        sumk+=1
res= sumk/(1.0*len(x_final_test))     

In [25]:
print("Accuracy is {}".format(res))

Accuracy is 0.93304347826087
